## This notebook contains instructions for setting up Spyglass data sharing through Kachery cloud.

### Overview of functionality
Spyglass uses Kachery Cloud (https://github.com/flatironinstitute/kachery-cloud) to make it possible to share analysis results (stored in NWB files) with others who do not have access to the filesystem where the original files are stored. 

When a user tries to access an AnalysisNWB file, Spyglass does the following:
1. Try to load it from the local store. 
2. If it is not available, check if the file is in the relevant sharing table (NwbKachery or AnalysisNWBKachery).
3. If so, attempt to download from the associated Kachery Resource.

Note that large file downloads may take a long time, so we currently do not support sharing of the main NWB files in this way. We suggest direct transfer of these files with globus or a similar service.

### Overview of setup

The Frank laboratory has three separate Kachery zones:
1. franklab.default         
        Used for all internal file sharing, including figurls
2. franklab.collaborator    
        Used for files that should be shared with collaborating labs.
3. franklab.public          
        Used for files that are publicly accessible (not yet active)

You may or may not want to set up a similar list on your systems depending on your sharing needs, and as each zone is setup in the same way, you can choose how many zones to establish or add new zones later relatively easily.



#### Setting up Kachery zones.

The instructions for setting up each zone can be found at https://github.com/flatironinstitute/kachery-cloud/blob/main/doc/create_kachery_zone.md 
<br>
<br>
These instructions walk you through the process of creating a cloud bucket and registering it with the Kachery team. Note that the bucket names cannot include periods, so we substite a dash, as in "franklab-default".

1. Follow the instructions for the first zone you want to create.
2. Follow the same instructions for the subsequent zones, but note that you can skip the 
Click "Create API Token" step

#### Setting up Kachery resources.

If you want to share files on demand from a zone, you need to set up a resource for that zone, following the instructions at https://github.com/scratchrealm/kachery-resource/blob/main/README.md

We suggest making the name of the zone and the resource identical to make things easy.

Note that for each zone, you need to run the local daemon that listens for requests from that zone. An example of the bash script we use is
```
#!/bin/bash
export KACHERY_ZONE=franklab.collaborators
export KACHERY_CLOUD_DIR=/stelmo/nwb/.kachery_cloud
cd /stelmo/nwb/franklab_collaborators_resource
npx kachery-resource@latest share


### Add zones/resources to your Spyglass database

Once the zones and resources exist, you need to add them to the database

First we import spyglass, etc. 

In [31]:
import spyglass as sg
from spyglass.common import Lab, AnalysisNwbfile
from spyglass.sharing import (KacheryZone, NwbfileKacherySelection, NwbfileKachery, 
                              AnalysisNwbfileKachery, AnalysisNwbfileKacherySelection)
import warnings
warnings.filterwarnings('ignore')

We start by inserting into the KacheryZone table, which will be empty if you're just starting out. 
change the entries in zone_key below to match your zone.

Note that the lab_name is a reference to the Lab() table, so get the name from there to match a lab that is defined already.

In [20]:
KacheryZone()

kachery_zone_name the name of the kachery zone. Note that this is the same as the name of the kachery resource.,description description of this zone,kachery_cloud_dir kachery cloud directory on local machine where files are linked,kachery_proxy kachery sharing proxy,lab_name
franklab.default,internal franklab kachery zone,/stelmo/nwb/.kachery_cloud,https://kachery-resource-proxy.herokuapp.com,Loren Frank


In [ ]:
zone_key = {'kachery_zone_name': 'franklab.collaborators', 
            'description'      : 'franklab collaborator zone',
            'kachery_cloud_dir': '/stelmo/nwb/.kachery_cloud',
            'kachery_proxy'    : 'https://kachery-resource-proxy.herokuapp.com',
            'lab_name'         : 'Loren Frank'}
KacheryZone().insert1(zone_key)

Now that our zone exists, all we have to do is add the AnalysisNWB files that we want to share. To do so, add entries to the AnalysisNwbfileKacherySelectionTable as sketched out below

In [14]:
#note that this would share all AnalysisFiles associated with a given nwb_file
nwb_file_name = "your_file_.nwb"

analysis_file_list = (AnalysisNwbfile() & {'nwb_file_name': nwb_file_name}).fetch('analysis_file_name')]

anks_key = {'kachery_zone_name': 'franklab.collaborators', 'analysis_file_name': ''}
for file in analysis_file_list:
    anks_key['analysis_file_name'] = file
    AnalysisNwbfileKacherySelection.insert1(anks_key)


NWB file /Users/loren/data/nwb/analysis/wilbur20210331/wilbur20210331_VZJAR6N0QM.nwb does not exist locally; checking kachery
uri: sha1://52995d7ffffcbcee8214dca923f2ba75fe6abb4c
File uploading


Now that we have those files in the selection table, we can add them as links to the kachery zone by populating the AnalysisNwbfileKachery table:

In [32]:
AnalysisNwbfileKachery.populate()

If all of that worked, 

1) go to https://kachery-gateway.figurl.org/admin?zone=your_zone
(changing your_zone to the name of your zone)
2) Go to the Admin/Authorization Settings tab
3) Add the github login names and permissions for the users you want to share with. 

If those users can connect to your database, they should now be able to use the .fetch_nwb() method to download any AnalysisNwbfiles that have been shared through Kachery.
